In [ ]:
"""
First version for training
"""

In [ ]:
"""
targets
    size: batch_size(15) x fi(52 = 50 + eos + bos) x fj(4)
"""

In [33]:
import os
import torch
import numpy as np
import math
import json
import time
import random
from collections import Counter, defaultdict
from sklearn.preprocessing import normalize
from torch import nn, tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer,\
    TransformerDecoder, TransformerDecoderLayer
from torch.nn.functional import softmax
from itertools import compress

In [46]:
d_model = 512
dropout = 0.1
max_len = 500
nhead = 8
encoder_layer_nums = 6
decoder_layer_nums = 6
dim_ff = 512
# token: ATCG(1~4), <BOS or SOS>(0), <EOS>(16), <PAD>(17)
    # combinations: AT, AC, AG, TC, TG, CG, ATC, ATG, ACG, TCG, ATCG(5~15, total 11)
ntoken = 18
alphabet_dict = {'A' : 1, 'T' : 2, 'C' : 3, 'G' : 4}

In [18]:
class TransformerModel(nn.Module): #done
    def __init__(
        self,
        d_model,
        dropout,
        max_len,
        nhead,
        encoder_layer_nums,
        decoder_layer_nums,
        dim_ff,
        ntoken,
    ):
        '''
        
        '''
        super(TransformerModel, self).__init__()
        
        self.d_model = d_model
        self.pos_encode = PositionalEmbedding(d_model, dropout, max_len)
        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_ff, dropout, batch_first = True)
        self.encoder = TransformerEncoder(encoder_layer, encoder_layer_nums)
        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_ff, dropout, batch_first = True)
        self.decoder = TransformerDecoder(decoder_layer, decoder_layer_nums)
        self.src_embedding = nn.Embedding(ntoken, d_model, scale_grad_by_freq = False)
        self.tgt_embedding = nn.Embedding(4, d_model, scale_grad_by_freq = False)
        self.out_embed = nn.Embedding(ntoken, d_model)
        self.output_linear = nn.Linear(d_model, ntoken)
        self.output_softmax = nn.Softmax(dim = -1)
        
        self.init_weights()

    
    def init_weights(self):
        initrange = 0.1
        self.src_embedding.weight.data.uniform_(-initrange, initrange)
        self.tgt_embedding.weight.data.uniform_(-initrange, initrange)
        self.output_linear.bias.data.zero_()
        self.output_linear.weight.data.uniform_(-initrange, initrange)
    
    def forward(
        self,
        src,
        tgt,
        src_mask = None,
        tgt_mask = None,
        memory_mask = None,
        src_key_padding_mask = None,
        tgt_key_padding_mask = None,
        memory_key_padding_mask = None,
    ):
        
        src = self.src_embedding(src)
        tgt = self.tgt_embedding(tgt)
        
        src = self.pos_encode(src)
        tgt = self.pos_encode(tgt)
        
        memory = self.encoder(src, mask = src_mask, src_key_padding_mask = src_key_padding_mask)
        output = self.decoder(tgt, memory, 
                              tgt_mask = tgt_mask, 
                              memory_mask = memory_mask, 
                              tgt_key_padding_mask = tgt_key_padding_mask,
                              memory_key_padding_mask = memory_key_padding_mask
                             )
        output = self.output_linear(output)
        output = self.output_softmax(output)
        
        return output

In [15]:
class PositionalEmbedding(nn.Module): #done
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 1000):
        super(PositionalEmbedding, self).__init__()
        self.dropout = nn.Dropout(p = dropout)
        
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp( torch.arange(0, d_model, 2) * (-math.log(10000) / d_model) )
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    
    def forward(self, x: Tensor):
        x = x + self.pe[0, :x.size(1), :].requires_grad_(False)
        output = self.dropout(x)
        return output
    

In [ ]:
model.train()
total_loss = 0.
times = 100

for step in range(1):
    data, target = get_batch()
    I = indicator(data)
    data_key_padding_mask = get_padding_mask(data)
    target_key_padding_mask = get_padding_mask(target)
    target_mask = nn.Transformer.generate_square_subsequent_mask(target.size(-1))
    last_f = None
    for _ in range(times):###
        output = model(data, target,
                       src_key_padding_mask = data_key_padding_mask,
                       tgt_key_padding_mask = target_key_padding_mask,
                       tgt_mask = target_mask
                      )
        if dist(last_f, output) < epsilon:
            break
        loss = EM_criterion(output, I, last_f)
        last_f = output
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        #torch.nn.utils.clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()
        

In [35]:
def indicator(data):
    assert isinstance(data, list), 'Input type for data is not list.'
    length = len(data[0])
    indicator = list()
    for seq in data:
        indicator.append(list(map(lambda x: alphabet_dict[x], seq)))
        assert len(seq) == length, 'The length is different for input seq'

    return tensor(indicator, dtype = torch.long)

In [ ]:
def indicator_list_ver(data):
    assert isinstance(data, list), 'Input type for data is not list.'
    length = len(data[0])
    indicator = list()
    for seq in data:
        indicator.append( torch.tensor(list(map(lambda x: alphabet_dict[x], seq)) ))
        assert len(seq) == length, 'The length is different for input seq'

    return indicator

In [ ]:
def fij_x_indicator(f, I):
    '''
    
    '''
    return

In [ ]:
def compute_prob():
    

In [ ]:
def EM_criterion(new_output, I, last_output = None, sub_len, last_lambda):
    assert new_output.shape[0] == last_output.shape[0],\
                'The batch size for last and new output unequal.'
    batch_size = new_output.shape[0]
    if torch.is_tensor(I):
        I = list(I)

    likelihood, nll, f, lamb = [], [], [], []
    for batch in range(batch_size):
        likelihood_res, nll_res, new_f, new_lamb = EM_step(I, batch, last_lamb, new_output[batch])
        likelihood.append(likelihood_res)
        nll.append(nll_res)
        f.append(new_f)
        lamb.append(new_lamb)
    
    return torch.sum(nll)

def EM_step(I, batch, last_lamb, f_ij, update_f = True):

    assert torch.is_tensor(last_lamb), "The lambda value is not in tensor(EM_step function)."
    f_ij = torch.log(f_ij)
    W = 10 + batch
    px1 = list()
    px2 = list()
    n = 0

    for seq in I:
        for i in range(len(seq) - W):
            n += 1
            p1, p2 = 0, 0
            for j in range(W):
                p1 += f_ij[j][seq[i+j]]
                p2 += f_ij[0][seq[i+j]]
            px1.append(p1)
            px2.append(p2)

    lamb = last_lamb.tile(n,1).t() #make the lambda value into 2xn shape
    prob = tensor([px1,px2])
    multi = torch.multiply(prob, lamb)
    Z = torch.div(multi, torch.sum(multi, dim = 0)) # shape: 2xn

    # nll = negative log likelihood function
    likelihood = Z * torch.log(prob) + Z * torch.log(lamb)
    nll = -likelihood

    new_lamb = Z / n

    if update_f:
        c0 = torch.zeros(1,4)
        cj = torch.zeros(W,4)
        for seq in I:
            for i in range(len(seq) - W):
                for j in range(W):
                    for L in range(4):
                        c0[0][L] += Z[1][i] if seq[i+j] == (L + 1) else 0
                    cj[j][seq[i+j] - 1] += Z[0][i]
        c0_sum = torch.sum(c0, axis = 1)
        cj_sum = torch.sum(cj, axis = 1)
        new_f0 = c0 / c0_sum
        new_fij = torch.zeros(W,4)
        for i in range(W):
            new_fij[i] = cj[i]/cj_sum
    
    
    return likelihood, nll, new_f, new_lamb

def condi_distribution(f, I, n, W):
    '''
    Calculate the conditional distribution p(Xi | theta_j)
    eq(7),(8) in the MEME article
    To avoid the computation error for the digits, it will use log and exponential
    
    Arguments:
    
    freq_letter: the frequences for each letter in each position, size: (W + 1) x L
                 background ( 1 x L ) + motif ( W x L )
                 dtype: np.array
    ====================================================
    return: it will be the log form output
    
    p_Xi_1: conditional distribution of motif sequence, size: n x 1
    p_Xi_2: conditional distribution of background, size: n x 1
    '''
    
    p_Xi_1 = torch.zeros(n)
    p_Xi_2 = torch.zeros(n)
    f = torch.log(f)
    f_0 = f[0]
    f_j = f[1:]
    
    for subseq in range(n):
        for pos in range(W):
            p_Xi_1[subseq] += f_j[pos][I[subseq][pos] - 1]
            p_Xi_2[subseq] += f_0[I[subseq][pos] - 1]

    return torch.exp(p_Xi_1), torch.exp(p_Xi_2)

In [36]:
def get_batch():
    """
    preprocess the data into the specific form
    """

    batch_size = 15
    W = 30
    dna_seq = ['GGGGATGCCGT','CTTAATGCTAC']
    alphabet_dict = {'A' : 1, 'T' : 2, 'C' : 3, 'G' : 4}
    #dna_seq = read_file(os.path.join(fileDir, file_list[ith]))
    #assert len(dna_seq) < batch_size, f' The (row) size of the input data is out of range(>{batch_size}).'
    row = len(dna_seq)
    #target = make_outputs(dna_seq, batch_size)
    #data = np.zeros((row, 500))
    #for idx in range(len(dna_seq)):
    #    print(f'doing the {idx} step in for loop')
    #    data[idx] = preprocess_seq(dna_seq[idx], 1)
        
    #data = expand_dim(data, 1)

    row, col = len(dna_seq), len(dna_seq[0])
    data = torch.zeros((row, col))
    
    for i in range(row):
        for j in range(col):
            data[i][j] = alphabet_dict[dna_seq[i][j]]
    col_expand = 16 * torch.ones((row,100 - col))
    row_expand = 16 * torch.ones((batch_size - row, 100))
    data = torch.cat( (torch.cat((data, col_expand), dim = 1), row_expand), dim = 0)

    target = torch.zeros((batch_size, W))
    for i in range(batch_size):
        target[i][1:i+11] += torch.ones(i+10)
        target[i][i+11] = 3
        target[i][i+12:W] += 2 * torch.ones(W - 2 - i - 10)
    
    return data, target

In [37]:
def get_padding_mask(data, pad_num = 2):
    '''
    create the padding mask for data
    return:
    mask: the padding mask in Tensor, with size same as the input data for the function
    '''
    mask = torch.zeros(data.shape)
    mask[data == pad_num] = -torch.inf
    
    return mask

In [25]:
src, tgt = get_batch()
src = src.to(torch.int64)
tgt = tgt.to(torch.int64)
model = TransformerModel(d_model, dropout, max_len, nhead, encoder_layer_nums, decoder_layer_nums, dim_ff, ntoken)

In [ ]:
model(torch.LongTensor(src),torch.LongTensor(tgt))

In [41]:
a = torch.arange(10).reshape(2,5)
print(a.size())

torch.Size([2, 5])


In [52]:
dna = ['GGGGATGCCGT','CTTAATGCTAC']

In [56]:
tmp = torch.arange(10)
print(tmp)
tmp.tolist()
print(tmp)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
